<a href="https://colab.research.google.com/github/huyhoang1996vn/ai_application/blob/main/ai_for_dev_B5_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!pip install pandas


In [63]:
posts = [{
    "name": "Iphone 15",
    "description": "iPhone 15 và 15 Plus được Apple giới thiệu vào ngày 12 tháng 9 năm 2023, tại Sự kiện Apple ở Apple Park. Chúng có thiết kế mới với mặt kính sau phủ màu, Dynamic Island, camera chính 48MP với ống kính Telephoto 2x và USB-C. Các điện thoại được ra mắt với iOS 17 và tương thích với iOS 18.",
    "price": 20000000,
    "discount": "20%"
},
        {
    "name": "Iphone 14",
    "description": "iPhone 14 và iPhone 14 Plus được giới thiệu vào ngày 7 tháng 9 năm 2022, là iPhone thế hệ thứ 16, kế thừa dòng iPhone 13. Chúng có hai kích thước: màn hình 6,1 inch và màn hình 6,7 inch mới, lớn hơn. Các tính năng chính bao gồm hệ thống camera kép mới, Phát hiện va chạm và SOS khẩn cấp qua vệ tinh, lần đầu tiên có trên điện thoại thông minh.",
    "price": 19000000,
    "discount": "15%"
},
          {
    "name": "Iphone 13",
    "description": "Màn hình Always On là tính năng mới của iPhone 13 được mong chờ bậc nhất. Tận dụng đặc tính không hao pin ở những nơi không hiển thị hình ảnh của tấm nền OLED",
    "price": 19000000,
    "discount": "13%"
},
           {
    "name": "Iphone 12",
    "description": "Điểm nổi bật của các dòng iPhone 12 là khả năng hỗ trợ kết nối 5G, tiết kiệm năng lượng và cung cấp tốc độ Internet nhanh hơn. Bên cạnh đó, chúng còn được trang bị con chip A14 Bionic mạnh mẽ, màn hình Super Retina XDR OLED xịn sò và hệ thống camera cao cấp (phiên bản Pro), đem lại trải nghiệm tuyệt vời cho người dùng",
    "price": 12000000,
    "discount": "12%"
}
         ]

In [64]:
import pandas as pd
df = pd.DataFrame(posts)
print("==== df ", df.head())
print("==== df ", df)

==== df          name                                        description     price  \
0  Iphone 15  iPhone 15 và 15 Plus được Apple giới thiệu vào...  20000000   
1  Iphone 14  iPhone 14 và iPhone 14 Plus được giới thiệu và...  19000000   
2  Iphone 13  Màn hình Always On là tính năng mới của iPhone...  19000000   
3  Iphone 12  Điểm nổi bật của các dòng iPhone 12 là khả năn...  12000000   

  discount  
0      20%  
1      15%  
2      13%  
3      12%  
==== df          name                                        description     price  \
0  Iphone 15  iPhone 15 và 15 Plus được Apple giới thiệu vào...  20000000   
1  Iphone 14  iPhone 14 và iPhone 14 Plus được giới thiệu và...  19000000   
2  Iphone 13  Màn hình Always On là tính năng mới của iPhone...  19000000   
3  Iphone 12  Điểm nổi bật của các dòng iPhone 12 là khả năn...  12000000   

  discount  
0      20%  
1      15%  
2      13%  
3      12%  


In [65]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('keepitreal/vietnamese-sbert')

def get_embedding(text: str):
    # Load the pre-trained model

    # Encode sentences into embeddings
    embeddings = model.encode(text)

    return embeddings.tolist()

In [66]:
df["embedding"] = df["description"].apply(get_embedding)

In [12]:
df.to_dict("records")

[{'name': 'Iphone 15',
  'description': 'The iPhone 15 and 15 Plus were introduced by Apple on September 12, 2023, at the Apple Event at Apple Park. They feature a new design with color-infused back glass, a Dynamic Island, a 48MP Main camera with 2x Telephoto, and USB-C. The phones launched with iOS 17 and are compatible with iOS 18. ',
  'price': 20000000,
  'embedding': [-0.21935825049877167,
   -0.17872501909732819,
   -0.154471293091774,
   -0.0742611512541771,
   -0.22861242294311523,
   0.011425099335610867,
   -0.15148532390594482,
   0.17598974704742432,
   -0.07427660375833511,
   0.02273721806704998,
   0.056648507714271545,
   -0.3434102237224579,
   0.1919942945241928,
   0.16344042122364044,
   -0.07544534653425217,
   -0.0435870997607708,
   -0.017354384064674377,
   -0.08190575987100601,
   -0.18840546905994415,
   -0.2289162129163742,
   0.09804830700159073,
   0.06198406219482422,
   -0.05761308595538139,
   -0.18117941915988922,
   0.37269043922424316,
   -0.06305295

In [49]:
!pip install pymongo

In [67]:
from pymongo import MongoClient
from google.colab import userdata
mongo_url = userdata.get('mongo')
client = MongoClient(mongo_url)
db = client["mycollection"]
collection = db['data_embedding']
records = df.to_dict("records")
# insert_result = collection.insert_many(records)

In [68]:
# https://www.mongodb.com/docs/atlas/atlas-vector-search/tutorials/local-rag/
def vector_search(user_query, collection, limit=4):
    query_embedding = get_embedding(user_query)
    vector_search_stage = {
        "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": 400,
            "limit": limit,
            # "score": {
            #     "$meta": "vectorSearchScore"
            # }
        }
    }
    unset_stage = {
        "$unset": "embedding"
    }

    project_stage = {
        "$project": {
            "_id": 0,
            "name": 1,
            "description": 1,
            "price": 1,
            "discount": 1,
            "score": {
                "$meta": "vectorSearchScore"
            }
        }
    }
    pipeline = [vector_search_stage, unset_stage, project_stage]
    result = collection.aggregate(pipeline)
    return list(result)


In [69]:
def get_search_result(query: str, collection: str) -> str:
    """Search for products in a collection and return formatted results.

    Args:
        query: Search query string
        collection: Name of the collection to search in

    Returns:
        Formatted string with product information
    """
    get_knowledge = vector_search(query, collection, 10)
    search_result = ""
    i = 0

    for result in get_knowledge:
        print(result)

        if result.get('name'):
            i += 1
            search_result += f"\n{i}. Tên: {result.get('name')}"

            # Add price information
            if result.get('price'):
                search_result += f", Giá: {result.get('price')}"
            else:
                # Mock up data
                search_result += ", Giá: Liên hệ để trao đổi thêm!"

            # Add promotion if available
            if result.get('discount'):
                search_result += f", Ưu đãi giảm: {result.get('discount')}%"

            if result.get('description'):
                search_result += f", Đặc điểm: {result.get('description')}%"

    return search_result

In [70]:
query = "iphone 12 5G chip A14 Bionic mạnh mẽ"
source_info = get_search_result(query.lower(), collection).replace('<br>', '\n')
print(source_info)

{'name': 'Iphone 12', 'description': 'Điểm nổi bật của các dòng iPhone 12 là khả năng hỗ trợ kết nối 5G, tiết kiệm năng lượng và cung cấp tốc độ Internet nhanh hơn. Bên cạnh đó, chúng còn được trang bị con chip A14 Bionic mạnh mẽ, màn hình Super Retina XDR OLED xịn sò và hệ thống camera cao cấp (phiên bản Pro), đem lại trải nghiệm tuyệt vời cho người dùng', 'price': 12000000, 'discount': '12%', 'score': 0.7715296745300293}
{'name': 'Iphone 14', 'description': 'iPhone 14 và iPhone 14 Plus được giới thiệu vào ngày 7 tháng 9 năm 2022, là iPhone thế hệ thứ 16, kế thừa dòng iPhone 13. Chúng có hai kích thước: màn hình 6,1 inch và màn hình 6,7 inch mới, lớn hơn. Các tính năng chính bao gồm hệ thống camera kép mới, Phát hiện va chạm và SOS khẩn cấp qua vệ tinh, lần đầu tiên có trên điện thoại thông minh.', 'price': 19000000, 'discount': '15%', 'score': 0.741421639919281}
{'name': 'Iphone 15', 'description': 'iPhone 15 và 15 Plus được Apple giới thiệu vào ngày 12 tháng 9 năm 2023, tại Sự kiện 

In [71]:
combined_infomation = f"Hãy trở thành chuyên gia tư vấn bán điện thoại cho một cửa hàng. Câu hỏi của khách hàng {query.lower()}. Trả lời câu hỏi dựa vào các thông tin sau: {source_info}"
print(combined_infomation)

Hãy trở thành chuyên gia tư vấn bán điện thoại cho một cửa hàng. Câu hỏi của khách hàng iphone 12 5g chip a14 bionic mạnh mẽ. Trả lời câu hỏi dựa vào các thông tin sau: 
1. Tên: Iphone 12, Giá: 12000000, Ưu đãi giảm: 12%%, Đặc điểm: Điểm nổi bật của các dòng iPhone 12 là khả năng hỗ trợ kết nối 5G, tiết kiệm năng lượng và cung cấp tốc độ Internet nhanh hơn. Bên cạnh đó, chúng còn được trang bị con chip A14 Bionic mạnh mẽ, màn hình Super Retina XDR OLED xịn sò và hệ thống camera cao cấp (phiên bản Pro), đem lại trải nghiệm tuyệt vời cho người dùng%
2. Tên: Iphone 14, Giá: 19000000, Ưu đãi giảm: 15%%, Đặc điểm: iPhone 14 và iPhone 14 Plus được giới thiệu vào ngày 7 tháng 9 năm 2022, là iPhone thế hệ thứ 16, kế thừa dòng iPhone 13. Chúng có hai kích thước: màn hình 6,1 inch và màn hình 6,7 inch mới, lớn hơn. Các tính năng chính bao gồm hệ thống camera kép mới, Phát hiện va chạm và SOS khẩn cấp qua vệ tinh, lần đầu tiên có trên điện thoại thông minh.%
3. Tên: Iphone 15, Giá: 20000000, Ưu đ

In [72]:

import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('.', '*')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [73]:
from google.colab import userdata
gemini_key = userdata.get('gemini')
genai.configure (api_key=gemini_key)
model = genai.GenerativeModel('gemini-1.5-pro')



In [61]:
response = model.generate_content(combined_infomation)
to_markdown(response.text)

> Chào anh/chị! Anh/chị đang quan tâm đến iPhone 12 với chip A14 Bionic mạnh mẽ đúng không ạ? Chip A14 Bionic quả thực là một điểm mạnh của iPhone 12, mang đến hiệu năng xử lý rất tốt ở thời điểm hiện tại*  Hiện tại iPhone 12 bên em đang có chương trình giảm giá 12%, chỉ còn 10*560*000đ thôi ạ* Với mức giá này, anh/chị sẽ sở hữu một chiếc điện thoại hỗ trợ 5G, tốc độ internet nhanh, màn hình Super Retina XDR OLED sắc nét và hệ thống camera chất lượng*
> 
> Tuy nhiên, để anh/chị có thể lựa chọn được chiếc điện thoại phù hợp nhất với nhu cầu, em xin phép được tư vấn thêm một vài lựa chọn khác:
> 
> * **Nếu anh/chị ưu tiên hiệu năng và ngân sách hợp lý:** iPhone 12 là một lựa chọn rất đáng cân nhắc*
> 
> * **Nếu anh/chị muốn trải nghiệm màn hình lớn hơn và các tính năng an toàn mới:** iPhone 14 và 14 Plus với màn hình 6*1 inch và 6*7 inch, cùng tính năng Phát hiện va chạm và SOS khẩn cấp sẽ là lựa chọn tốt hơn*  Hiện tại iPhone 14 đang được giảm giá 15% còn 16*150*000đ*
> 
> * **Nếu anh/chị muốn sở hữu công nghệ mới nhất với camera 48MP, Dynamic Island và cổng USB-C:** iPhone 15 và 15 Plus là sự lựa chọn hàng đầu* Hiện tại iPhone 15 đang được giảm giá 20% còn 16*000*000đ*
> 
> * **Nếu anh/chị thích tính năng màn hình Always-On:** iPhone 13 cũng là một lựa chọn đáng cân nhắc với mức giá 16*430*000đ sau khi giảm 13%*
> 
> Tóm lại, mỗi dòng iPhone đều có những ưu điểm riêng*  Anh/chị có thể chia sẻ thêm về nhu cầu sử dụng, ví dụ như hay dùng để chụp ảnh, chơi game, xem phim,*** để em có thể tư vấn cụ thể hơn giúp anh/chị lựa chọn được sản phẩm ưng ý nhất*


In [20]:
!pip install -qU semantic-router

In [21]:
# https://github.com/aurelio-labs/semantic-router
from semantic_router import Route

product_route = Route(
    name="product",
    utterances=[
    "Bạn có sẵn iPhone mới nhất không?",
    "Giá của Samsung Galaxy S21 là bao nhiêu?",
    "OnePlus 9 Pro có màu xanh không?",
    "Thông số kỹ thuật của Google Pixel 6 là gì?",
    "Huawei P50 Pro có sẵn ở cửa hàng của bạn không?",
    "iPhone 13 có những màu gì?",
    "Có giảm giá nào cho Samsung Galaxy Note 20 không?",
    "Sony Xperia 1 III có sẵn không?",
    "Cửa hàng của bạn có Google Pixel mới nhất không?",
    "Có chương trình khuyến mãi nào cho OnePlus Nord không?",
    "Dung lượng lưu trữ của Samsung Galaxy S21 là bao nhiêu?",
    "iPhone SE 2022 có sẵn không?",
    "Sự khác biệt về giá giữa iPhone 12 và 13 là gì?",
    "Motorola Edge 20 có sẵn không?",
    "Google Pixel 6 Pro có sẵn ở cửa hàng của bạn không?",
    "Bạn có bán Xiaomi Mi 11 không?",
    "Có chương trình ưu đãi nào cho Samsung Galaxy Z Fold 3 không?",
    "Oppo Find X3 Pro có sẵn không?",
    "Các tính năng của iPhone 13 Pro là gì?",
    "LG Wing có sẵn không?",
    "Nokia 8.3 5G có sẵn không?",
    "Asus ROG Phone 5 có sẵn không?",
    "Bạn có Realme GT không?",
    "Vivo X60 Pro có sẵn không?",
    "Bạn có Honor 50 không?",
    "Tuổi thọ pin của Samsung Galaxy A52 là bao nhiêu?",
    "ZTE Axon 30 có sẵn không?",
    "Bạn có BlackBerry KEY2 không?",
    "Kích thước màn hình của iPhone 13 Mini là bao nhiêu?",
    "Bạn có TCL 20 Pro 5G không?",
    "Nokia XR20 có sẵn ở cửa hàng của bạn không?",
    "Giá của Samsung Galaxy S20 FE là bao nhiêu?",
    "Bạn có iPhone 12 Pro Max không?",
    "Redmi Note 10 Pro có sẵn không?",
    "Sự khác biệt giữa iPhone 12 và iPhone 13 là gì?",
    "Bạn có Sony Xperia 5 II không?",
    "Thời gian bảo hành của Samsung Galaxy S21 là bao lâu?",
    "Google Pixel 5a có sẵn không?",
    "Bạn có OnePlus 8T không?",
    "Giá của iPhone 13 Pro Max là bao nhiêu?",
    "Samsung Galaxy Z Flip 3 có sẵn không?",
    "Thông số kỹ thuật của Oppo Reno6 Pro là gì?",
    "Bạn có Vivo V21 không?",
    "Motorola Moto G100 có sẵn không?",
    "Bạn có Huawei Mate 40 Pro không?",
    "Realme 8 Pro có sẵn ở cửa hàng của bạn không?",
    "Asus Zenfone 8 có sẵn không?",
    "LG Velvet có sẵn không?",
    "Dung lượng lưu trữ của iPhone 12 là bao nhiêu?",
    "Bạn có Honor Magic 3 không?",
    "Xiaomi Mi 11 Ultra có sẵn không?"
],
)
chitchat_route = Route(
    name="chitchat",
    utterances=[
    "Thời tiết hôm nay như thế nào?",
    "Ngoài trời nóng bao nhiêu?",
    "Ngày mai có mưa không?",
    "Nhiệt độ hiện tại là bao nhiêu?",
    "Bạn có thể cho tôi biết điều kiện thời tiết hiện tại không?",
    "Cuối tuần này có nắng không?",
    "Nhiệt độ hôm qua là bao nhiêu?",
    "Đêm nay trời sẽ lạnh đến mức nào?",
    "Ai là tổng thống đầu tiên của Hoa Kỳ?",
    "Chiến tranh thế giới thứ hai kết thúc vào năm nào?",
    "Bạn có thể kể cho tôi về lịch sử của internet không?",
    "Tháp Eiffel được xây dựng vào năm nào?",
    "Ai đã phát minh ra điện thoại?",
    "Tên của bạn là gì?",
    "Bạn có tên không?",
    "Tôi nên gọi bạn là gì?",
    "Ai đã tạo ra bạn?",
    "Bạn bao nhiêu tuổi?",
    "Bạn có thể kể cho tôi một sự thật thú vị không?",
    "Bạn có biết bất kỳ câu đố thú vị nào không?",
    "Màu sắc yêu thích của bạn là gì?",
    "Bộ phim yêu thích của bạn là gì?",
    "Bạn có sở thích nào không?",
    "Ý nghĩa của cuộc sống là gì?",
    "Bạn có thể kể cho tôi một câu chuyện cười không?",
    "Thủ đô của Pháp là gì?",
    "Dân số thế giới là bao nhiêu?",
    "Có bao nhiêu châu lục?",
    "Ai đã viết 'Giết con chim nhại'?",
    "Bạn có thể cho tôi một câu nói của Albert Einstein không?"
],
)
routes = [product_route, chitchat_route]

In [22]:
from semantic_router.encoders import OpenAIEncoder
import os
from google.colab import userdata
gpt_key = userdata.get('gpt')
os.environ["OPENAI_API_KEY"] = gpt_key

encoder = OpenAIEncoder()

In [27]:
from semantic_router.routers import SemanticRouter

route_layer = SemanticRouter(encoder=encoder, routes=routes, auto_sync="local")

In [29]:
user_input = "Iphone 12 bao nhiêu?"
output = route_layer(user_input)
print("output", output)
if output.name == "product":
    print("The query is about product.")
elif output.name == "chitchat":
    print("The query is about chitchat.")
else:
    print("The query does not match any defined routes.")

output name='product' function_call=None similarity_score=None
The query is about product.


In [31]:
!pip install openai

In [32]:
historyString = "Iphone 12 còn hàng không? giá bao nhiêu? có khuyến mãi không? Bên mình có giảm giá không ạ? có tính năng gì ạ"
higherLevelSummariesPrompt = """Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question in Vietnamese which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is. {historyString}
        """.format(historyString=historyString)

In [33]:
from google.colab import userdata
from openai import OpenAI

gpt_key = userdata.get('gpt')
client = OpenAI(api_key=gpt_key)
response = client.responses.create(
    model="gpt-4o-mini",
    input=higherLevelSummariesPrompt
)

print(response.output_text)

Iphone 12 còn hàng không? Giá bao nhiêu? Có khuyến mãi hay giảm giá gì không? Nó có những tính năng gì?
